In [ ]:
import subprocess
import tempfile

import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
                
# parallel processor
from utilities import parallel_process, check_exe, system_call
from utilities import get_immediate_subdirectories, get_immediate_subfiles

In [ ]:
# this allows us to run the pipeline not interactively. 
os.environ['HEADASNOQUERY'] = ''
os.environ['HEADASPROMPT'] = '/dev/null'

In [ ]:
def run_pipeline(name, outpath, overwrite=False):

    if not os.path.isdir(f'{outpath}/{name}'):
        return
    
    if not check_exe('xrtpipeline', verb=False):
        return
    
    # get a list of the XRT obs. 
    obs = get_immediate_subdirectories(f'{outpath}/{name}')
        
    # if there aren't any observations, keep going
    if not len(obs):
        return
    
    if not os.path.isdir(f'{outpath}/{name}/reduced'):
        os.makedirs(f'{outpath}/{name}/reduced')
    
    # get a list of the reduced obs.
    reduc = get_immediate_subdirectories(f'{outpath}/{name}/reduced')
    
    for ob_dir in obs:
        ob_id = ob_dir.split('/')[-1]
        reduc_ids = [r.split('/')[-1] for r in reduc]
        evts = False
        expm = False
        if ob_id == 'reduced':
            continue
        elif ob_id == 'spec_files':
            continue
        else:
            try:
                for f in get_immediate_subfiles(f'{outpath}/{name}/reduced/{ob_id}'):
                    if 'po_cl.evt' in f:
                        evts = True
                    elif 'po_ex.img' in f:
                        expm = True
                    else:
                        continue    
                if evts and expm and not overwrite:
                    continue
            except FileNotFoundError: # happens when nothing has been reduced
                pass
                    
        cmd = (f'xrtpipeline indir={ob_dir} outdir={outpath}/{name}/reduced/{ob_id} steminputs=sw{ob_id} '
                    'srcra=OBJECT srcdec=OBJECT datamode=PC cleanup=yes vigflag=yes clobber=yes ')
        
        stdout, stderr = system_call_env(cmd, shlexify=False)
        
        # log the output
        log_file = f'{outpath}/{name}/reduced/{ob_id}_reduce.log'
        with open(log_file, 'w') as f:
            f.writelines(stdout)

    return

In [ ]:
def validate():
    '''This has been rolled into the main function above''' 
    
    PSZs = get_immediate_subdirectories('./data_full')
    for psz in tqdm_notebook(PSZs):
        # get a list of the XRT obs. 
            for ob_dir in get_immediate_subdirectories(psz):
                # check for events and exposure map
                evts = False
                expm = False
                if ob_id == 'reduced':
                    continue
                else:
                    for f in get_immediate_subfiles(f'{psz}/reduced/{ob_id}'):
                        if 'po_cl.evt' in f:
                            evts = True
                        elif 'po_ex.img' in f:
                            expm = True
                        else:
                            continue
                
                    if not evts and expm:
                        print(f'{psz}/reduced/{ob_id} NOT VALID')

In [ ]:
# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

outpath = '../data_full'

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath, 'overwrite':False} for n in data['NAME']]
parallel_process(arr, run_pipeline, use_kwargs=True, n_jobs=6)


In [ ]:
# outpath = '../data_full'
# name = 'PSZ2_G110.28-87.48'
# run_pipeline(name, outpath, False)